In [33]:
import re
import os
import sys

def loadModel(fileName):
    with open(fileName, "rb") as f:
        if fileName.endswith(".py"):
            return eval(f.read()) #eval() 函数用来执行一个字符串表达式，并返回表达式的值
        elif fileName.endswith(".txt"):
            result = set()
            for line in f:
                result.add(line.strip().decode('utf-8'))
            return result

prob_start = loadModel("./fileNeeded/prob_start.py") #dict
prob_trans = loadModel("./fileNeeded/prob_trans.py") #dict
prob_emit = loadModel("./fileNeeded/prob_emit.py") #dict
near_char_tab = loadModel("./fileNeeded/near_char_tab.txt") #set

def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}] #储存每个时刻的概率的列表
    path = {} #储存到4个隐藏状态的最优路径
    #t=0时刻（初始状态）
    for y in states: 
        V[0][y] = start_p[y] * emit_p[y].get(obs[0],0)
        path[y] = [y]
    #t=1,...,len(obs)-1时刻
    for t in range(1,len(obs)):
        V.append({})
        newpath = {}
        #当前时刻所处的各种可能的状态
        for y in states:
            #得到 最大概率 和 对应的前一个隐藏状态
            (prob,state) = max([ (V[t-1][y0] * trans_p[y0].get(y,0) * emit_p[y].get(obs[t],0), y0) 
                                            for y0 in states if V[t-1][y0]>0])
            #更新 t时刻 对应的隐藏状态为 y 的概率
            V[t][y] = prob
            #更新 新的路径 为 上一时刻最优的隐藏状态 + 这一时刻的隐藏状态
            newpath[y] = path[state] + [y]
        #更新上一次的路径
        path = newpath
    #选取 最大概率 和 对应的最后一个隐藏状态
    (prob, state) = max([(V[len(obs) - 1][y], y) for y in states])
    #返回最大概率和最优路径
    return (prob, path[state])

def __raw_seg(sentence):
    i,j =0,0
    while j<len(sentence)-1:
        if not ( sentence[j:j+2] in near_char_tab):
            yield sentence[i:j+1]
            i=j+1
        j+=1
    yield sentence[i:j+1]


def __cut(sentence):
    prob, pos_list =  viterbi(sentence,('B','M','E','S'), prob_start, prob_trans, prob_emit)# 通过viterbi算法求出隐藏状态序列
    begin, next = 0,0
    # print pos_list, sentence
    # 基于隐藏状态序列进行分词    
    for i,char in enumerate(sentence):
        pos = pos_list[i]
        if pos=='B':# 字所处的位置是开始位置
            begin = i
        elif pos=='E':# 字所处的位置是结束位置
            yield sentence[begin:i+1]# 这个子序列就是一个分词
            next = i+1
        elif pos=='S': # 单独成字
            yield char
            next = i+1
    if next<len(sentence): # 剩余的直接作为一个分词，返回
        yield sentence[next:]

def cut(sentence,find_new_word=False):
    re_han, re_skip = re.compile("([\u4E00-\u9FA5]+)"), re.compile("[^a-zA-Z0-9+#\n]")
    blocks = re_han.split(sentence)
    if find_new_word: 
        detail_seg = lambda x: (x,)
    else:
        detail_seg = __raw_seg
    for blk in blocks:
        if re_han.match(blk):
            for lb in detail_seg(blk):
                for word in __cut(lb):
                    yield word
        else:
            tmp = re_skip.split(blk)
            for x in tmp:
                if x!="":
                    yield x
sentence_list = [
"百度新闻是包含海量资讯的新闻服务平台,真实反映每时每刻的新闻热点",
"它主要是通过他人视角来聚焦戴安娜去世后的7天，深挖戴安娜备受英国民众追捧的原因，以及她去世后英国皇室和大众的关系。",
"迪迦奥特曼现身了"
]

print( "-----------默认效果-----------\n")

for sentence in sentence_list:
    seg_list = cut(sentence)
    print("/ ".join(seg_list))

print("\n-----------打开新词发现功能后的效果-----------\n")

for sentence in sentence_list:
    seg_list = cut(sentence,find_new_word=True)
    print("/ ".join(seg_list))

-----------默认效果-----------

百度/ 新闻/ 是/ 包含/ 海量/ 资讯/ 的/ 新闻/ 服务/ 平台/ 真实/ 反映/ 每时/ 每刻/ 的/ 新闻/ 热点
它/ 主要/ 是/ 通过/ 他/ 人/ 视角/ 来/ 聚焦/ 戴安娜/ 去/ 世后/ 的/ 7/ 天/ 深挖/ 戴安娜/ 备受/ 英国民众/ 追/ 捧/ 的/ 原因/ 以及/ 她/ 去/ 世后/ 英国/ 皇室/ 和/ 大众/ 的/ 关系
迪/ 迦/ 奥特/ 曼/ 现身/ 了

-----------打开新词发现功能后的效果-----------

百度/ 新闻/ 是/ 包含/ 海量/ 资讯/ 的/ 新闻/ 服务/ 平台/ 真实/ 反映/ 每时/ 每刻/ 的/ 新闻/ 热点
它/ 主要/ 是/ 通过/ 他/ 人视/ 角来/ 聚焦/ 戴/ 安娜/ 去/ 世后/ 的/ 7/ 天/ 深挖/ 戴/ 安娜/ 备受/ 英国民众/ 追/ 捧/ 的/ 原因/ 以及/ 她/ 去/ 世后/ 英国/ 皇室/ 和/ 大众/ 的/ 关系
迪迦/ 奥特曼/ 现身/ 了


## 备注1：Python dict的get方法
dict.get(key, default=None)：返回指定键的值，如果值不在字典中返回默认值。
例子：

In [35]:
testdict = {'Name': 'Zara', 'Age': 27}

print("Value : %s" %  testdict.get('Age'))
print("Value : %s" %  testdict.get('Sex', "Never"))

Value : 27
Value : Never


## 备注2：合并两个list的操作 - 直接相加
在viterbi算法的实现中有用到：``` newpath[y] = path[state] + [y] ```这行代码，其中相关的语法就是：将两个list直接相加就可以将两者合并

In [9]:
ab=['a','b']
c=['c']
print(ab+c)

['a', 'b', 'c']


## 备注3：max函数
在viterbi算法实现中有用到：
```
max([ (V[t-1][y0] * trans_p[y0].get(y,0) * emit_p[y].get(obs[t],0), y0)  for y0 in states if V[t-1][y0]>0])
```
这行代码，这里的max函数看上去虽然是对元组进行比较，但是实际上它只比较元组中的第一个元素，以下面的例子为例。

这里max函数返回的是 a这个list中第一个元素（数值类型）最大的那个元组

In [10]:
a=[(2,'b'),(1,'a')]
print(max(a))

(2, 'b')
